In [61]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

from numpy import array
from keras.preprocessing.text import one_hot
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers import Embedding
from keras.layers import LSTM
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

In [62]:
df_train = pd.read_csv("./drugsComTrain_raw.csv")
df_test = pd.read_csv("./drugsComTest_raw.csv")

print("Train shape :" ,df_train.shape)
print("Test shape :", df_test.shape)

merge = [df_train, df_test]
df_data = pd.concat(merge)

print (df_data.shape)

df_data.head(10)

Train shape : (161297, 7)
Test shape : (53766, 7)
(215063, 7)


,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37
5,155963,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2,28-Nov-15,43
6,165907,Levonorgestrel,Emergency Contraception,"""He pulled out, but he cummed a bit in me. I t...",1,7-Mar-17,5
7,102654,Aripiprazole,Bipolar Disorde,"""Abilify changed my life. There is hope. I was...",10,14-Mar-15,32
8,74811,Keppra,Epilepsy,""" I Ve had nothing but problems with the Kepp...",1,9-Aug-16,11
9,48928,Ethinyl estradiol / levonorgestrel,Birth Control,"""I had been on the pill for many years. When m...",8,8-Dec-16,1


In [63]:
print ("Null values in the dataset : ", df_data.isnull().sum(axis = 0))

Null values in the dataset :  uniqueID          0
drugName          0
condition      1194
review            0
rating            0
date              0
usefulCount       0
dtype: int64


In [64]:
df_data = df_data.dropna(how = 'any', axis = 0)

print ("The shape of the dataset after null values removal :", df_data.shape)

The shape of the dataset after null values removal : (213869, 7)


In [65]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [66]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [67]:
X = []
sentences = list(df_data['review'])
for sen in sentences:
    X.append(preprocess_text(sen))
    

In [68]:
df_data['sentiment'] = df_data["rating"].apply(lambda x: 1 if x > 5 else 0)
y = list(df_data['sentiment'])

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [70]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [71]:
vocab_size = len(tokenizer.word_index) + 1

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [72]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('D:/Recommendation/glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [73]:
embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [85]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [79]:
model = Sequential()
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model.add(embedding_layer)
model.add(LSTM(128))

model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [80]:
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 100, 100)          4681300   
                                                                 
 lstm_4 (LSTM)               (None, 128)               117248    
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 4,798,677
Trainable params: 117,377
Non-trainable params: 4,681,300
_________________________________________________________________
None


In [81]:
history = model.fit(X_train, y_train, batch_size=128, epochs=6)

Epoch 1/6
1337/1337 [==============================] - 250s 186ms/step - loss: 0.5399 - acc: 0.7390
Epoch 2/6
1337/1337 [==============================] - 247s 184ms/step - loss: 0.4029 - acc: 0.8197
Epoch 3/6
1337/1337 [==============================] - 247s 185ms/step - loss: 0.3535 - acc: 0.8472
Epoch 4/6
1337/1337 [==============================] - 254s 190ms/step - loss: 0.3228 - acc: 0.8622
Epoch 5/6
1337/1337 [==============================] - 253s 189ms/step - loss: 0.2990 - acc: 0.8728
Epoch 6/6
1337/1337 [==============================] - 253s 189ms/step - loss: 0.2761 - acc: 0.8846


In [86]:
score = model.evaluate(X_test, y_test, verbose=1)

1337/1337 [==============================] - 34s 25ms/step - loss: 0.2972 - acc: 0.8735


In [88]:
df_data.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount,sentiment
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,1
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,1
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,0
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10,1
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37,1


In [89]:
X = tokenizer.texts_to_sequences(X)
X = pad_sequences(X, padding='post', maxlen=maxlen)

In [90]:
sub_preds_deep = model.predict(X)

6684/6684 [==============================] - 170s 25ms/step


In [93]:
sub_preds_deep.shape

(213869, 1)

In [94]:
df_data['dl_pred'] = sub_preds_deep

In [95]:
df_data.to_csv('file2.csv')